In [1]:
from plotlib import load_ring, load_mod, export_pi_ring, export_pi_mod
from plotlib import *

In [2]:
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\benchmark\S0_AdamsSS_t100.db'
path_mod = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\benchmark\Cnu_AdamsSS_t100.db'

In [3]:
# data = load_ring(path_ring)
# export_pi_ring(data)

In [4]:
data_ring, data_mod = load_mod(path_ring,path_mod)
export_pi_mod(data_ring, data_mod)

In [5]:
# radius = get_radius(data_ring)
# radius1 = get_radius(data_mod)
# tpl_bullets, index2xyrp = export_bullets([data_ring, data_mod], [radius, radius1])
# lines, dashed_lines = export_basis_prod(data_ring, index2xyrp, 0)
# lines1, dashed_lines1 = export_basis_prod(data_mod, index2xyrp, len(data_ring["basis"]))